# data download MAC/MON

- https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE117970
- Human Tumor-Associated Macrophage and Monocyte Transcriptional Landscapes Reveal Cancer-Specific Reprogramming, Biomarkers, and Therapeutic Targets


In [174]:
library(DESeq2)
library(data.table)
library(dplyr)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(EnsDb.Hsapiens.v75)
library(clusterProfiler)
library(R.utils)
library(AnnotationDbi)
library(org.Hs.eg.db)
library(ggplot2)
library(gridExtra)
library(IRdisplay)
library(GGally)
library(ChIPpeakAnno)
library(IRdisplay)
library(ggpubr)
library(DiffBind)
library(Seurat)
library(kableExtra)
library(tidyverse)
library(ggraph)
library(tidygraph)
library(clusterProfiler)
library(enrichplot)
library(readxl)
library(umap)
library(corrplot)
library(ggrepel)
library(msigdbr)
library(fgsea)
library(ggvenn)
library(ComplexHeatmap)
library(ggVennDiagram)
library(venneuler)
library(sva)
library(DT)
library(tools)
library(tidyverse)
library(maSigPro)
library(RSQLite)
library(glue)

library(edgeR)

txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene


In [175]:
data_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/'
base_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC/' 
setwd(base_dir)


# organizze IDs from GEO database

In [397]:
dsn_gse117970_string = glue(
'GSM3316272	MON2: Normal	Monocyte_normal_rep1
GSM3316273	MON4: Normal	Monocyte_normal_rep1
GSM3316274	MON8: Normal	Monocyte_normal_rep1
GSM3316275	MON10: Normal	Monocyte_normal_rep1
GSM3316276	MON12: Breast cancer	Monocyte_BRCA_rep1
GSM3316277	MON14: Breast cancer	Monocyte_BRCA_rep1
GSM3316278	MON18: Breast cancer	Monocyte_BRCA_rep1
GSM3316279	MON20: Breast cancer	Monocyte_BRCA_rep1
GSM3316280	MON22: Endometrial cancer	Monocyte_CaEnd_rep1
GSM3316281	MON24: Endometrial cancer	Monocyte_CaEnd_rep1
GSM3316282	MON26: Endometrial cancer	Monocyte_CaEnd_rep1
GSM3316283	MON28: Endometrial cancer	Monocyte_CaEnd_rep1
GSM3316284	MON30: Endometrial cancer	Monocyte_CaEnd_rep1
GSM3316285	MON57: Normal	Monocyte_BRCA_rep2
GSM3316286	MON59: Breast cancer	Monocyte_BRCA_rep2
GSM3316287	MON61: Breast cancer	Monocyte_BRCA_rep2
GSM3316288	MON63: Endometrial cancer	Monocyte_CaEnd_rep2
GSM3316289	MON65: Endometrial cancer	Monocyte_CaEnd_rep2
GSM3316290	MON66: Normal	Monocyte_normal_rep2
GSM3316291	MON67: Normal	Monocyte_normal_rep2
GSM3316292	MON68: Normal	Monocyte_normal_rep2
GSM3316293	MON69: Normal	Monocyte_normal_rep2
GSM3316294	MON70: Normal	Monocyte_normal_rep2
GSM3316295	MON71: Normal	Monocyte_normal_rep2
GSM3316296	MON72: Normal	Monocyte_normal_rep2
GSM3316297	MON73: Normal	Monocyte_normal_rep2
GSM3316298	MON74: Normal	Monocyte_normal_rep2
GSM3316299	MON75: Normal	Monocyte_normal_rep2
GSM3316300	MON76: Breast cancer	Monocyte_BRCA_rep2
GSM3316301	MON78: Breast cancer	Monocyte_BRCA_rep2
GSM3316302	MON79: Endometrial cancer	Monocyte_CaEnd_rep2
GSM3316303	MON80: Endometrial cancer	Monocyte_CaEnd_rep2
GSM3316304	MON81: Breast cancer	Monocyte_BRCA_rep2
GSM3316305	MON82: Breast cancer	Monocyte_BRCA_rep2
GSM3316306	MON83: Breast cancer	Monocyte_BRCA_rep2
GSM3316307	MON84: Breast cancer	Monocyte_BRCA_rep2
GSM3316308	MON85: Breast cancer	Monocyte_BRCA_rep2
GSM3316309	MON87: Breast cancer	Monocyte_BRCA_rep2
GSM3316310	MON103: Normal	Monocyte_normal_rep3
GSM3316311	MON104: Normal	Monocyte_normal_rep3
GSM3316312	MON105: Normal	Monocyte_normal_rep3
GSM3316313	MON106: Normal	Monocyte_normal_rep3
GSM3316314	MON107: Normal	Monocyte_normal_rep3
GSM3316315	MON93: Breast cancer	Monocyte_BRCA_rep3
GSM3316316	MON94: Breast cancer	Monocyte_BRCA_rep3
GSM3316317	MON95: Breast cancer	Monocyte_BRCA_rep3
GSM3316318	MON96: Breast cancer	Monocyte_BRCA_rep3
GSM3316319	MON97: Breast cancer	Monocyte_BRCA_rep3
GSM3316320	MON118: Breast cancer	Monocyte_BRCA_rep3
GSM3316321	MON119: Breast cancer	Monocyte_BRCA_rep3
GSM3316322	MON120: Breast cancer	Monocyte_BRCA_rep3
GSM3316323	MON121: Breast cancer	Monocyte_BRCA_rep3
GSM3316324	MON112: Endometrial cancer	Monocyte_CaEnd_rep3
GSM3316325	MAC3: Normal Breast	Macrophage_Breast_rep1
GSM3316326	MAC4: Normal Breast	Macrophage_Breast_rep1
GSM3316327	MAC5: Normal Breast	Macrophage_Breast_rep1
GSM3316328	MAC9: Breast cancer	Macrophage_BRCA_rep1
GSM3316329	MAC12: Breast cancer	Macrophage_BRCA_rep1
GSM3316330	MAC14: Normal Endometrial	Macrophage_Endometrial_rep1
GSM3316331	MAC16: Normal Endometrial	Macrophage_Endometrial_rep1
GSM3316332	MAC17: Normal Endometrial	Macrophage_Endometrial_rep1
GSM3316333	MAC19: Endometrial cancer	Macrophage_CaEnd_rep1
GSM3316334	MAC21: Endometrial cancer	Macrophage_CaEnd_rep1
GSM3316335	MAC22: Endometrial cancer	Macrophage_CaEnd_rep1
GSM3316337	MAC23: Endometrial cancer	Macrophage_CaEnd_rep1
GSM3316339	MAC24: Endometrial cancer	Macrophage_CaEnd_rep1
GSM3316342	MAC25: Endometrial cancer	Macrophage_CaEnd_rep1
GSM3316345	MAC27: Breast cancer	Macrophage_BRCA_rep1
GSM3316347	MAC28: Normal Breast	Macrophage_Breast_rep1
GSM3316350	MAC29: Breast cancer	Macrophage_BRCA_rep1
GSM3316352	MAC30: Normal Endometrial	Macrophage_Breast_rep2
GSM3316353	MAC31: Normal Endometrial	Macrophage_Breast_rep2
GSM3316354	MAC32: Endometrial cancer	Macrophage_CaEnd_rep2
GSM3316355	MAC33: Endometrial cancer	Macrophage_CaEnd_rep2
GSM3316356	MAC35: Endometrial cancer	Macrophage_CaEnd_rep2
GSM3585969	mon103	Monocyte_normal_rep4
GSM3585970	mon104	Monocyte_normal_rep4
GSM3585971	mon105	Monocyte_normal_rep4
GSM3585972	mon106	Monocyte_normal_rep4
GSM3585973	mon107	Monocyte_normal_rep4
GSM3585974	mon108	Monocyte_normal_rep4
GSM3585975	mon109	Monocyte_normal_rep4
GSM3585976	mon110	Monocyte_normal_rep4
GSM3585977	mon111	Monocyte_normal_rep4
GSM3585978	mon112	Monocyte_normal_rep4
GSM3585979	mon113	Monocyte_normal_rep4
GSM3585980	mon114	Monocyte_normal_rep5
GSM3585981	mon115	Monocyte_normal_rep5
GSM3585982	mon116	Monocyte_normal_rep5
GSM3585983	mon117	Monocyte_normal_rep5
GSM3585984	mon118	Monocyte_normal_rep5
GSM3585985	mon127	Monocyte_normal_rep5
GSM3585986	mon128	Monocyte_normal_rep5
GSM3585987	mon129	Monocyte_normal_rep5
GSM3585988	mon130	Monocyte_normal_rep5
GSM3585989	mon131	Monocyte_normal_rep5
GSM3585990	mon132	Monocyte_normal_rep5
GSM3585991	mon133	Monocyte_normal_rep5
GSM3585992	mon134	Monocyte_normal_rep5
GSM3585993	mon135	Monocyte_normal_rep6
GSM3585994	mon136	Monocyte_normal_rep6
GSM3585995	mon137	Monocyte_normal_rep6
GSM3585996	mon138	Monocyte_normal_rep6
GSM3585997	mon139	Monocyte_normal_rep6
GSM3585998	mon140	Monocyte_normal_rep6
GSM3585999	mon141	Monocyte_normal_rep6
GSM3586000	mon142	Monocyte_normal_rep6
GSM3586001	mon144	Monocyte_normal_rep6
GSM3586002	mon145	Monocyte_normal_rep6
GSM3586003	mon146	Monocyte_normal_rep6
GSM3586004	mon147	Monocyte_normal_rep6
GSM3586005	mon155	Monocyte_normal_rep6
GSM3586006	mon156	Monocyte_normal_rep6
GSM3586007	mon157	Monocyte_normal_rep6
GSM3586008	mon158	Monocyte_normal_rep6
GSM3586009	mon159	Monocyte_normal_rep6')


In [192]:
# DC1, DC2, pDC
dsn_gse76511_string = glue(
'GSM2027310	1005_D60_Flu10_DC1	CD1cDC_normal_rep1
GSM2027311	1005_D60_Flu10_DC2	CD141DC_normal_rep1
GSM2027312	1005_D60_Flu10_DC3	BDCA2CD2NpDC_normal_rep1
GSM2027313	1005_D60_Flu10_DC4	BDCA2CD2PpDC_normal_rep1
GSM2027314	1006_D60_Flu10_DC1	CD1cDC_normal_rep2
GSM2027315	1006_D60_Flu10_DC2	CD141DC_normal_rep2
GSM2027316	1006_D60_Flu10_DC3	BDCA2CD2NpDC_normal_rep2
GSM2027317	1006_D60_Flu10_DC4	BDCA2CD2PpDC_normal_rep2
GSM2027318	1009_D60_Flu10_DC1	CD1cDC_normal_rep3
GSM2027319	1009_D60_Flu10_DC2	CD141DC_normal_rep3
GSM2027320	1009_D60_Flu10_DC3	BDCA2CD2NpDC_normal_rep3
GSM2027321	1009_D60_Flu10_DC4	BDCA2CD2PpDC_normal_rep3'
)


In [193]:
#Total RNA sequencing was performed on CD141+ DC isolated from peripheral blood of healthy individuals (n=4), 
# synovial fluid of patients with inflammatory arthritis (n=5) and peripheral blood of patients with inflammatory arthritis (n=7). Cd1c+ DC were also sequenced from peripheral blood of healthy individuals (n=3).
dsn_gse1081174_string = glue(
'GSM2891732	healthy peripheral 2 (CD141+ DC)	CD141DC_normal_rep2
GSM2891733	healthy peripheral 4 (CD141+ DC)	CD141DC_normal_rep2
GSM2891734	healthy peripheral 5 (CD141+ DC)	CD141DC_normal_rep2
GSM2891735	healthy peripheral 6 (CD141+ DC)	CD141DC_normal_rep2
GSM2891748	healthy peripheral 1 (Cd1c+ DC)	CD1cDC_normal_rep2
GSM2891749	healthy peripheral 2 (Cd1c+ DC)	CD1cDC_normal_rep2
GSM2891750	healthy peripheral 3 (Cd1c+ DC)	CD1cDC_normal_rep2'
)

#
# GSM2891736	IA synovial 1 (CD141+ DC)
# GSM2891737	IA synovial 2 (CD141+ DC)
# GSM2891738	IA synovial 3 (CD141+ DC)
# GSM2891739	IA synovial 4 (CD141+ DC)
# GSM2891740	IA synovial 6 (CD141+ DC)
# GSM2891741	IA peripheral 1 (CD141+ DC)
# GSM2891742	IA peripheral 2 (CD141+ DC)
# GSM2891743	IA peripheral 3 (CD141+ DC)
# GSM2891744	IA peripheral 4 (CD141+ DC)
# GSM2891745	IA peripheral 5 (CD141+ DC)
# GSM2891746	IA peripheral 6 (CD141+ DC)
# GSM2891747	IA peripheral 7 (CD141+ DC)



In [396]:
dsn_string = list(dsn_gse117970_string,  dsn_gse76511_string,  dsn_gse1081174_string) 
dsn_list = lapply(dsn_string, function(xx){
    dsn_dt = fread(text = xx, sep = '\t')
    dsn_dt = as.data.table(dsn_dt)
    dsn_dt = setnames(dsn_dt, c('gsm_id', 'assay_sample_desc', 'sample_name'))
})
names(dsn_list) = c('GSE117970', 'GSE76511', 'GSE1081174')
dsn_dt = rbindlist(dsn_list, idcol = T)


ERROR: Error in eval(expr, envir, enclos): object 'dsn_gse119954_string' not found


In [195]:
dsn_dt[, assay_sample_name := sub('(.*)_.*', '\\1', sample_name)]
dsn_dt[, assay_sample_name := paste0(.BY$assay_sample_name, '_rep', 1:nrow(.SD)), by = 'assay_sample_name']
head(dsn_dt)


.id,gsm_id,assay_sample_desc,sample_name,assay_sample_name
<chr>,<chr>,<chr>,<chr>,<chr>
GSE117970,GSM3316273,MON4: Normal,Monocyte_normal_rep1,Monocyte_normal_rep1
GSE117970,GSM3316274,MON8: Normal,Monocyte_normal_rep1,Monocyte_normal_rep2
GSE117970,GSM3316275,MON10: Normal,Monocyte_normal_rep1,Monocyte_normal_rep3
GSE117970,GSM3316276,MON12: Breast cancer,Monocyte_BRCA_rep1,Monocyte_BRCA_rep1
GSE117970,GSM3316277,MON14: Breast cancer,Monocyte_BRCA_rep1,Monocyte_BRCA_rep2
GSE117970,GSM3316278,MON18: Breast cancer,Monocyte_BRCA_rep1,Monocyte_BRCA_rep3


# download fastqs 

In [196]:
gsmid = fread(paste0(data_dir, 'gsmid'), header = F)
setnames(gsmid, c('gsm_id', 'srr_id'))

In [197]:
head(gsmid)

gsm_id,srr_id
<chr>,<chr>
gsm_id,srr_id
GSM3316273,"SRR7631577,SRR7631578,"
GSM3316274,"SRR7631579,"
GSM3316275,"SRR7631580,SRR7631581,"
GSM3316276,"SRR7631582,"
GSM3316277,"SRR7631583,"


In [198]:
#fwrite(gsmid, sep = '\t', file = 'gsmid')

In [199]:
dsn_dt2 = merge(dsn_dt, gsmid, by = 'gsm_id', all.x = T) 

In [200]:
dsn_dt3 = dsn_dt2 %>% 
  group_by(srr_id) %>% 
  separate_rows(srr_id, sep = ",") %>% 
  filter(srr_id != '') %>% as.data.table()

In [201]:
head(dsn_dt3)

gsm_id,.id,assay_sample_desc,sample_name,assay_sample_name,srr_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2027311,GSE76511,1005_D60_Flu10_DC2,CD141DC_normal_rep1,CD141DC_normal_rep1,SRR3084022
GSM2027312,GSE76511,1005_D60_Flu10_DC3,BDCA2CD2NpDC_normal_rep1,BDCA2CD2NpDC_normal_rep1,SRR3084023
GSM2027313,GSE76511,1005_D60_Flu10_DC4,BDCA2CD2PpDC_normal_rep1,BDCA2CD2PpDC_normal_rep1,SRR3084024
GSM2027314,GSE76511,1006_D60_Flu10_DC1,CD1cDC_normal_rep2,CD1cDC_normal_rep1,SRR3084025
GSM2027315,GSE76511,1006_D60_Flu10_DC2,CD141DC_normal_rep2,CD141DC_normal_rep2,SRR3084026
GSM2027316,GSE76511,1006_D60_Flu10_DC3,BDCA2CD2NpDC_normal_rep2,BDCA2CD2NpDC_normal_rep2,SRR3084027


In [202]:
dsn_dt4[is.na(sra_file), ]

srr_id,gsm_id,.id,assay_sample_desc,sample_name,assay_sample_name,cmd,sra_file,assay_sample_dir,prefetch_cmd,fastqdump_cmd,dump_r,dump_r1,dump_r2,gz_r_cmd,gz_r1_cmd,gz_r2_cmd,r_fastq_gz,r1_fastq_gz,r2_fastq_gz
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [203]:
# find SRR ID
dsn_dt4[, cmd := paste0('echo "', gsm_id, '\n" && ', '~/edirect/esearch -db sra -query "', gsm_id, '" | ~/edirect/efetch -format docsum | ~/edirect/accn-at-a-time | grep -i srr | tr [:lower:] [:upper:] | tr "\n" "," ')]
#dsn_dt[, srr_id := {system(,cmd, intern = T)}, by = 1:nrow(dsn_dt)]
write(dsn_dt4[!file.exists(sra_file), cmd], file = 'runid2.sh')
dsn_dt4[!file.exists(sra_file), cmd]
#fwrite(dsn_gse231486, file = 'dsn_gse231486.tsv')


character(0)

In [204]:
dsn_dt3[assay_sample_name %in% dsn_dt3[duplicated(assay_sample_name), assay_sample_name], ]

gsm_id,.id,assay_sample_desc,sample_name,assay_sample_name,srr_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM3316273,GSE117970,MON4: Normal,Monocyte_normal_rep1,Monocyte_normal_rep1,SRR7631577
GSM3316273,GSE117970,MON4: Normal,Monocyte_normal_rep1,Monocyte_normal_rep1,SRR7631578
GSM3316275,GSE117970,MON10: Normal,Monocyte_normal_rep1,Monocyte_normal_rep3,SRR7631580
GSM3316275,GSE117970,MON10: Normal,Monocyte_normal_rep1,Monocyte_normal_rep3,SRR7631581
GSM3316279,GSE117970,MON20: Breast cancer,Monocyte_BRCA_rep1,Monocyte_BRCA_rep4,SRR7631585
GSM3316279,GSE117970,MON20: Breast cancer,Monocyte_BRCA_rep1,Monocyte_BRCA_rep4,SRR7631586
GSM3316280,GSE117970,MON22: Endometrial cancer,Monocyte_CaEnd_rep1,Monocyte_CaEnd_rep1,SRR7631587
GSM3316280,GSE117970,MON22: Endometrial cancer,Monocyte_CaEnd_rep1,Monocyte_CaEnd_rep1,SRR7631588
GSM3316283,GSE117970,MON28: Endometrial cancer,Monocyte_CaEnd_rep1,Monocyte_CaEnd_rep4,SRR7631591


In [205]:
tmp = fread(paste0(data_dir, 'sra_list3'), header = F)
tmp[, sra_file := normalizePath(paste0(data_dir, V1))]
tmp[, base_sra_file := basename(V1) ] 
tmp[, srr_id := sub('.sra', '', base_sra_file)]
dsn_dt4 = merge(dsn_dt3, tmp[, .(srr_id, sra_file)], by = 'srr_id', all.x = T)


In [206]:
dim(dsn_dt3)
dim(dsn)

[1] 140   6

[1] 140  19

In [222]:
dsn_dt4[, assay_sample_dir := paste0(data_dir, assay_sample_name, '/')]

In [ ]:
inv = dsn[, {dir.create(assay_sample_dir, showWarnings = T, recursive = F)}, by = 1:nrow(dsn)] 
all(inv)

In [249]:
dsn_dt4[, prefetch_cmd := paste0('prefetch ',     srr_id, ' -O ', assay_sample_dir, ' --max-size 40g')]
dsn_dt4[, fastqdump_cmd:= paste0('cd ', assay_sample_dir, ' && fasterq-dump ', srr_id, ' --outdir ', assay_sample_dir, ' & ')]

# if single end seq
dsn_dt4[, dump_r := paste0(assay_sample_dir,  '/', srr_id, '.fastq')]
# if double end seq
dsn_dt4[, dump_r1 := paste0(assay_sample_dir, '/', srr_id, '_1.fastq')] 
dsn_dt4[, dump_r2 := paste0(assay_sample_dir, '/', srr_id, '_2.fastq')]

dsn_dt4[, gz_r_cmd := paste0(' gzip ', dump_r, ' &' )]
dsn_dt4[, gz_r1_cmd := paste0(' gzip ', dump_r1, ' &' )]
dsn_dt4[, gz_r2_cmd := paste0(' gzip ', dump_r2, ' &' )]

dsn_dt4[, r_fastq_gz := paste0(dump_r, '.gz')]
dsn_dt4[, r1_fastq_gz := paste0(dump_r1, '.gz')]
dsn_dt4[, r2_fastq_gz := paste0(dump_r2, '.gz')]


In [233]:
head(dsn_dt4[, sra_file])
head(dsn_dt4[, new_sra_file])

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/SRR3084022/SRR3084022.sra"     
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/SRR3084023/SRR3084023.sra"
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2PpDC_normal_rep1/SRR3084024/SRR3084024.sra"
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD1cDC_normal_rep1/SRR3084025/SRR3084025.sra"      
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/SRR3084026/SRR3084026.sra"     
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/SRR3084027/SRR3084027.sra"

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/SRR3084022/SRR3084022.sra"     
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/SRR3084023/SRR3084023.sra"
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2PpDC_normal_rep1/SRR3084024/SRR3084024.sra"
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD1cDC_normal_rep1/SRR3084025/SRR3084025.sra"      
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep2/SRR3084026/SRR3084026.sra"     
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep2/SRR3084027/SRR3084027.sra"

In [234]:
dsn_dt4[, cmd := paste0('mv ', sra_file, ' ', new_sra_file)]

In [236]:
dsn_dt4$cmd[5]
dsn_dt4$assay_sample_dir[5]


[1] "mv /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/SRR3084026/SRR3084026.sra /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep2/SRR3084026/SRR3084026.sra"

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep2/"

In [243]:
dsn_dt4[, new_sra_file := paste0(assay_sample_dir, srr_id, '/', srr_id, '.sra')]
dsn_dt4[, new_r1_fastq_gz := paste0(assay_sample_dir, srr_id, '_1.fastq.gz')]
dsn_dt4[, new_r2_fastq_gz := paste0(assay_sample_dir, srr_id, '_2.fastq.gz')]


In [ ]:
dsn_dt4[, {system(paste0('mv ', sra_file, ' ', new_sra_file))}, by = 1:nrow(dsn_dt4)]

In [244]:
file.exists(dsn_dt4$new_r1_fastq_gz)

[1]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [13] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [25]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE
 [37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [61] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE
 [73] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [85]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE
 [97] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[109] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[121] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE

In [248]:
getwd()

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC"

In [246]:
dim(dsn_dt4[!file.exists(new_r1_fastq_gz), ])

[1] 99 23

In [255]:
write(dsn_dt4[, prefetch_cmd],  file = 'run_fastq_prefetch.sh')
write(dsn_dt4[!file.exists(new_r1_fastq_gz), fastqdump_cmd], file = 'run_fastq_dump.sh')
#write(dsn_dt4[file.exists(dump_r),  gz_r_cmd],  file = 'run_gzip.sh')
write(dsn_dt4[file.exists(dump_r1), gz_r1_cmd], file = 'run_gzip.sh', append = F)
write(dsn_dt4[file.exists(dump_r2), gz_r2_cmd], file = 'run_gzip.sh', append = T)


In [216]:
getwd()

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC"

In [209]:
dsn_dt4[, sra_base_name := basename(sra_file) ]


In [258]:
file.exists(dsn_dt4$new_r1_fastq_gz)
head(dsn_dt4$new_r1_fastq_gz)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[136] TRUE TRUE TRUE TRUE TRUE

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/SRR3084022_1.fastq.gz"     
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/SRR3084023_1.fastq.gz"
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2PpDC_normal_rep1/SRR3084024_1.fastq.gz"
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD1cDC_normal_rep1/SRR3084025_1.fastq.gz"      
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep2/SRR3084026_1.fastq.gz"     
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep2/SRR3084027_1.fastq.gz"

In [274]:
getwd()

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC"

In [242]:
tmp = copy(dsn_dt4)[!duplicated(sample_name), ]
tmp[, cell_type := unlist(strsplit(sample_name, '_'))[1], by = 1:nrow(tmp)]
tmp[, tissue := unlist(strsplit(sample_name, '_'))[2], by = 1:nrow(tmp)]
table(tmp$cell_type, tmp$tissue)


               
                BRCA Breast CaEnd Endometrial normal
  BDCA2CD2NpDC     0      0     0           0      2
  BDCA2CD2NpDCs    0      0     0           0      1
  BDCA2CD2PpDC     0      0     0           0      2
  BDCA2CD2PpDCs    0      0     0           0      1
  CD141DC          0      0     0           0      3
  CD1cDC           0      0     0           0      2
  Macrophage       1      2     2           1      0
  Monocyte         3      0     3           0      6

In [257]:
file.exists(dsn_dt4$sra_file)

[1]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [13] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [25]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE
 [37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [61] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE
 [73] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [85]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE
 [97] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[109] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[121] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE

In [70]:
dsn_dt4[assay_sample_name == 'CD141DC_normal_rep1',  .(assay_sample_name, sample_name)]

assay_sample_name,sample_name
<chr>,<chr>
CD141DC_normal_rep1,CD141DC_normal_rep1
CD141DC_normal_rep1,CD141DC_normal_rep2
CD141DC_normal_rep1,CD141DC_normal_rep3


In [259]:
dsn_dt4[duplicated(assay_sample_name), .(assay_sample_name, sample_name)] 

assay_sample_name,sample_name
<chr>,<chr>
Monocyte_normal_rep1,Monocyte_normal_rep1
Monocyte_normal_rep3,Monocyte_normal_rep1
Monocyte_BRCA_rep4,Monocyte_BRCA_rep1
Monocyte_CaEnd_rep1,Monocyte_CaEnd_rep1
Monocyte_CaEnd_rep4,Monocyte_CaEnd_rep1
Monocyte_CaEnd_rep5,Monocyte_CaEnd_rep1
Monocyte_BRCA_rep16,Monocyte_BRCA_rep3
Monocyte_BRCA_rep19,Monocyte_BRCA_rep3


In [66]:
head(dsn_dt4[, .(sample_dir, assay_sample_dir)])

sample_dir,assay_sample_dir
<chr>,<chr>
/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//CD141DC_normal_rep1/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/
/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//BDCA2CD2NpDC_normal_rep1/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/
/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//BDCA2CD2PpDC_normal_rep1/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2PpDC_normal_rep1/
/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//CD1cDC_normal_rep2/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD1cDC_normal_rep1/
/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//CD141DC_normal_rep2/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/
/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//BDCA2CD2NpDC_normal_rep2/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/


# merge the fastqs for a sample name, the original sequence sample


In [176]:
base_dir
setwd(base_dir)

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC/"

In [261]:
dsn_dt4[, nn := nrow(.SD), by = 'assay_sample_name']
dsn_dt4[, assay_sample_dir := paste0(data_dir, '/', assay_sample_name, '/')]
dsn_dt4[, r1_sample := paste0(assay_sample_dir, assay_sample_name, '_R1.fastq.gz')]
dsn_dt4[, r2_sample := paste0(assay_sample_dir, assay_sample_name, '_R2.fastq.gz')]

#inv = dsn[, {dir.create(sample_dir, showWarnings = F, recursive = T)}, by = 1:nrow(dsn)] 
#dsn_dt4[, {if(.SD$nn == 1 & file.exists(.SD$r2_fastq_gz) & file.size(.SD$r2_fastq_gz) > 1000){createLink(link = .SD$r2_sample, target = .SD$r2_fastq_gz)}}, by = 1:nrow(dsn)] 


In [262]:
dsn_dt4[, merge_r1_jobname := paste0('merge_r1_', sample_name)]
dsn_dt4[, merge_r1_oo := paste0(assay_sample_dir, merge_r1_jobname, '_out')]
dsn_dt4[, merge_r1_eo := paste0(assay_sample_dir, merge_r1_jobname, '_err')]
dsn_dt4[, merge_r1_cmd := paste0('bsub -J ', merge_r1_jobname, ' -n 2 -o ', merge_r1_oo, ' -e ', merge_r1_eo)]
dsn_dt4[, merge_r1_cmd := paste0(merge_r1_cmd, ' " cat ', paste(.SD$r1_fastq_gz, collapse = ' '), ' > ', r1_sample, '"'), by = 'assay_sample_name'] 

dsn_dt4[, merge_r2_jobname := paste0('merge_r2_', sample_name)]
dsn_dt4[, merge_r2_oo := paste0(assay_sample_dir, merge_r2_jobname, '_out')]
dsn_dt4[, merge_r2_eo := paste0(assay_sample_dir, merge_r2_jobname, '_err')]
dsn_dt4[, merge_r2_cmd := paste0('bsub -J ', merge_r2_jobname, ' -n 2 -o ', merge_r2_oo, ' -e ', merge_r2_eo)]
dsn_dt4[, merge_r2_cmd := paste0(merge_r2_cmd, ' " cat ', paste(.SD$r2_fastq_gz, collapse = ' '), ' > ', r2_sample, '"'), by = 'assay_sample_name'] 


In [280]:
inv = dsn_dt4[, {if(.SD$nn == 1){createLink(link = .SD$r1_sample, target = .SD$r1_fastq_gz)}}, by = 1:nrow(dsn_dt4)] 
inv = dsn_dt4[, {if(.SD$nn == 1){createLink(link = .SD$r2_sample, target = .SD$r2_fastq_gz)}}, by = 1:nrow(dsn_dt4)] 


In [309]:
nrow(dsn_dt4)

[1] 29

In [265]:
setwd(data_dir)
data_dir

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/"

In [266]:
write(dsn_dt4[nn > 1, merge_r1_cmd], file = 'run_rnaseq_merge_r1.sh')
write(dsn_dt4[nn > 1, merge_r2_cmd], file = 'run_rnaseq_merge_r2.sh')


In [267]:
file.size(dsn_dt4$r1_sample)

[1]  997993387  771067962 1132292117 1261441383 1008560867 1131785520
  [7] 1666922317  932848860  984274847 1128312323 1978765711 3014444627
 [13] 3484974382 3163734295 3215528489 2856587655 2914101902 2642678106
 [19] 2642678106 2203405815 4305959374 4305959374 3360903457 2896530164
 [25] 2582792419 5777177585 5777177585 2997184939 2997184939 2240135056
 [31] 4333706286 5226539562 5226539562 1937670266 1937670266 4162379459
 [37] 3473300608 2675358121 2190965069 4042275898 3826371430 4652267914
 [43] 4191866040 3019277431 3090998540 4005304465 2693739570 2480284742
 [49] 4538917309 3516057903 2762595809 5367398962 4137602581 3448282583
 [55] 3768335185 4364120320 4388572387 3761586785 3490114449 3372043830
 [61] 2105679180 2025722985 2027372450 2054096164 2029687862 2175465226
 [67] 2175465226 2133561515 2144920858 2234559147 2234559147 2148396492
 [73] 2154714540 2157053759 2188338005 2163482040 2122024184 3416668097
 [79] 4142034335 3705418118 4960238650 3486445158 2752433737 3764798858
 [85] 3367902287 3121347491 4413622910 3403779911 3932552248 5011149350
 [91] 5166509644 3670502188 2082532826 2128413822 2166797842 2085300311
 [97] 2123144552 2167911777 2004190406 3023054311 3836120486 3634692482
[103] 3531739319 3503573730 3872085054 2996623296 2728700834 2288239625
[109] 3657835598 3360764151 3666252691 3384897916 3460572030 3922384790
[115] 3976588654 2986489684 3381126994 3491853020 3460905435 3561655122
[121] 3184367560 3594279070 3145990325 2895568985 3524132493 2975908743
[127] 2782494180 2991448425 2752911013 3468020081 2931566354 3474715388
[133] 3008292569 2913579103 2861613268 3358827143 3371046093 3951345093
[139] 3084979133 3042183986

In [316]:
file.size(dsn_dt4$r1_sample)

[1]   997993387   771067962  1132292117 10247659429 10671714171  1131785520
 [7]  1666922317   932848860   984274847  1128312323  1978765711  5666385200
[13] 13474384796 13340833246 30721655120 13819126131 36015085344 10242558641
[19] 17432485510  2122024184 13346653376 14245599818  9885134882 25048961554
[25]  4252098153  6295246735 36433428886 45217363291 53388632050

In [268]:
dsn_dt5 = dsn_dt4[!duplicated(assay_sample_name), ]


# cutadapt

In [178]:
head(dsn_dt5[, .(sample_name, assay_sample_name, sample_dir, assay_sample_dir)])

sample_name,assay_sample_name,sample_dir,assay_sample_dir
<chr>,<chr>,<chr>,<chr>
CD141DC_normal_rep1,CD141DC_normal_rep1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//CD141DC_normal_rep1/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/
BDCA2CD2NpDC_normal_rep1,BDCA2CD2NpDC_normal_rep1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//BDCA2CD2NpDC_normal_rep1/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/
BDCA2CD2PpDC_normal_rep1,BDCA2CD2PpDC_normal_rep1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//BDCA2CD2PpDC_normal_rep1/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2PpDC_normal_rep1/
CD1cDC_normal_rep2,CD1cDC_normal_rep1,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//CD1cDC_normal_rep2/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD1cDC_normal_rep1/
CD141DC_normal_rep2,CD141DC_normal_rep2,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//CD141DC_normal_rep2/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep2/
BDCA2CD2NpDC_normal_rep2,BDCA2CD2NpDC_normal_rep2,/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC//BDCA2CD2NpDC_normal_rep2/,/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep2/


In [269]:
setwd(base_dir)


In [270]:
base_dir

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC/"

In [281]:
file.exists(dsn_dt5$r2_sample)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [273]:
dsn_dt5[, sample_dir := paste0(base_dir, sample_name, '/')]


In [291]:
# module load cutadapt/4.4
dsn_dt5[, fastq_r1_trimmed := paste0(assay_sample_dir, assay_sample_name, '_R1_trimmed.fa.gz')]
dsn_dt5[, fastq_r2_trimmed := paste0(assay_sample_dir, assay_sample_name, '_R2_trimmed.fa.gz')]
dsn_dt5[, cutadapt_jobname := paste0('cuta_', sample_name)]
dsn_dt5[, cutadapt_oo := paste0(assay_sample_dir, cutadapt_jobname, '_out')]
dsn_dt5[, cutadapt_eo := paste0(assay_sample_dir, cutadapt_jobname, '_err')]
dsn_dt5[, cutadapt_cmd := paste0('bsub -J ', cutadapt_jobname, ' -n 3 -o ', cutadapt_oo, ' -e ', cutadapt_eo)]
dsn_dt5[, cutadapt_cmd := paste0(cutadapt_cmd, ' " cutadapt -o ', fastq_r1_trimmed, ' -p ', fastq_r2_trimmed, ' ', r1_sample, ' ', r2_sample, '"')]
write(dsn_dt5[, cutadapt_cmd], file = paste0(data_dir, 'run_cutadapt.sh'))


In [282]:
file.exists(dsn_dt5$r2_sample)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [185]:
head(dsn_dt5$sample_dir)

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep1/"     
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep1/"
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2PpDC_normal_rep1/"
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD1cDC_normal_rep1/"      
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/CD141DC_normal_rep2/"     
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC/BDCA2CD2NpDC_normal_rep2/"

# align

In [334]:
file.exists(dsn_dt5$fastq_r2_trimmed)
file.size(dsn_dt5$fastq_r2_trimmed)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

[1]  972524561  737127571 1093753339 1189523050  955058816 1071295280
  [7] 1589427782  870224823  930484978 1057221300 1884379683 2837128323
 [13] 3300060942 2997486137 3038447544 2702115377 2747799743 2550344495
 [19] 2106534892 4151352287 3222512595 2797628998 2482013385 5521018206
 [25] 2895231369 2172875658 4125753178 5018468869 1847047374 4024432726
 [31] 3352855181 2547862197 2080691564 3839021691 3737258978 4561416048
 [37] 4102220447 2950203895 3012400130 3883649022 2613941160 2411111881
 [43] 4415154093 3408606138 2712881680 5268428390 4070234164 3397328933
 [49] 3577359085 4170614789 4203525610 3597849641 3378027601 3252060729
 [55] 1991769001 1911754673 1922539877 1949135562 1921511684 2072599347
 [61] 2018817503 2031040378 2115073865 2043371183 2056107653 2052503322
 [67] 2088198063 2058981944 2019958607 3355806403 4057777087 3625287332
 [73] 4856405758 3411310013 2687937385 3629169371 3256422979 3019444488
 [79] 4275668020 3232167671 3743760921 4754514233 4921422189 3536786834
 [85] 1974168443 2028378578 2059438867 1976133006 2010357060 2055633284
 [91] 1894988562 3260074283 4055984619 3891333266 3651587234 3696500326
 [97] 4112104550 3152401187 2844326025 2396154110 3771612860 3663892757
[103] 3708956579 3495562136 3650145919 3957054489 4072261297 3123397179
[109] 3487295755 3550001936 3673496656 3716583286 3300586602 3716410290
[115] 3261148374 3101824936 3439580638 3064126137 2903549322 3146015029
[121] 2840622909 3550038785 3039338491 3649250025 3078765749 3025638674
[127] 2987100559 3472373299 3501001249 4088435428 3275560147 3145456688

In [287]:
dim(dsn_dt5)

[1] 132  47

In [320]:
dsn_dt5[assay_sample_name == 'CD1cDC_normal_rep5', sample_name]

[1] "CD1cDC_normal_rep2"

In [311]:
# module load star/2.7.10a
REFERENCE_GENOME="/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF"
TE_GTF_FILE="/research/groups/jxugrp/home/common/Data_Public/RNA_seq/Demthylation_therapy_AML_cell_lines_RNA-seq_Khawanky_etal_Nature_communicat_GSE184891/trimmed_fastq/STAR_normal_mapped_t2t/TE_locus_analysis"

dsn_dt5[, star_jobname := paste0('star_', assay_sample_name)]
dsn_dt5[, star_oo := paste0(assay_sample_dir, 'star_', assay_sample_name, '_out')]
dsn_dt5[, star_eo := paste0(assay_sample_dir, 'star_', assay_sample_name, '_err')]
dsn_dt5[, bam_prefix := paste0(assay_sample_dir, assay_sample_name, '_')]
dsn_dt5[, bam_file := paste0(bam_prefix, 'Aligned.sortedByCoord.out.bam')]
dsn_dt5[, bam_tx_file := paste0(bam_prefix, 'Aligned.toTranscriptome.out.bam')]
#alignments/A549_0_1 Aligned.sortedByCoord.out.bam

dsn_dt5[, star_cmd := paste0('bsub -J ', star_jobname, ' -o ', star_oo, ' -e ', star_eo, ' -P macmon -n 25 -R "rusage[mem=5GB]" ')]
#dsn_dt5[, star_cmd := paste0(star_cmd, ' -w ', cutadapt_jobname)]
dsn_dt5[, star_cmd := paste0(star_cmd, ' -W 290')]
dsn_dt5[, star_cmd := paste0(star_cmd, ' " STAR --genomeDir ', REFERENCE_GENOME)] 
dsn_dt5[, star_cmd := paste0(star_cmd, ' --runThreadN 28 --outSAMtype BAM SortedByCoordinate --outFilterMultimapNmax 1000 --outSAMmultNmax 1 ')]
dsn_dt5[, star_cmd := paste0(star_cmd, ' --outFilterMismatchNmax 3 --outMultimapperOrder Random --winAnchorMultimapNmax 1000 --alignEndsType EndToEnd ')]
dsn_dt5[, star_cmd := paste0(star_cmd, ' --alignIntronMax 1 --alignMatesGapMax 350 --quantMode TranscriptomeSAM --readFilesIn ', fastq_r1_trimmed, ' ', fastq_r2_trimmed, ' ')]
#dsn_dt5[, star_cmd := paste0(star_cmd, ' --alignIntronMax 1 --alignMatesGapMax 350 --quantMode TranscriptomeSAM --readFilesIn ', fastq_r1_trimmed, ' ', fastq_r2_trimmed, ' ')]
dsn_dt5[, star_cmd := paste0(star_cmd, ' --readFilesCommand zcat --outFileNamePrefix ', bam_prefix, '"')]

write(dsn_dt5[, star_cmd], file = paste0(data_dir, 'run_star.sh'))

#STAR --genomeDir $REFERENCE_GENOME \
#                --runThreadN 28 --outSAMtype BAM SortedByCoordinate --outFilterMultimapNmax 1000 --outSAMmultNmax 1 \
#                --outFilterMismatchNmax 3 --outMultimapperOrder Random --winAnchorMultimapNmax 1000 --alignEndsType EndToEnd \
#                --alignIntronMax 1 --alignMatesGapMax 350 --quantMode TranscriptomeSAM --readFilesIn "$R1_FILE" "$R2_FILE" --readFilesCommand zcat \
#                --outFileNamePrefix "$OUTPUT_DIR/${SAMPLE_NAME}_"

In [318]:
file.exists(dsn_dt5[assay_sample_name == 'Monocyte_normal_rep47', bam_file ])
(dsn_dt5[assay_sample_name == 'Monocyte_normal_rep47', bam_file ])

[1] FALSE

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC//Monocyte_normal_rep47/Monocyte_normal_rep47_Aligned.sortedByCoord.out.bam"

In [324]:
file.exists(dsn_dt5$bam_file)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [300]:
dim(dsn_dt5)

[1] 132  51

In [21]:
write(dsn_sample[!file.exists(tecount_file), star_cmd], file = paste0(base_dir, 'run_star.sh'))


In [333]:
getwd()

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC"

# TEcount

In [335]:
file.size(dsn_dt5$bam_file)

[1] 1425707941 1169968874 1636129070 1877030204 1479541082 1749391351
  [7] 2555885633 1395192593 1461122578 1746275654 2988086615 2033678242
 [13] 2924563562 2167235327 3252200129 2886540806 2930361569 1242897323
 [19]  944519301 2061415039 1934167138 1642534326 1270550905 3467362120
 [25] 2413337826 1714581125 2777662544 3898874398  991382379 3805537263
 [31] 3665049297 2636102760 2326807172 3880869388 4013819409 4824779208
 [37] 4527787942 3383988970 3400387099 4238576909 2925288604 2706444683
 [43] 4464899364 3338274407 2876335032 5610176957 4415303885 3837759210
 [49] 3898006521 4714688049 4650664548 4220271722 3532274873 3593688908
 [55] 1994290243 1816797387 1845903270 1905388877 1798119542 1977668695
 [61] 2050606559 2029212476 2160654210 1989133682 1923287367 1891121800
 [67] 1961172485 1989180790 1838574962 3804250899 4462918540 3423202267
 [73] 4792851492 3439027218 2462715691 3833494607 3337649748 3038246084
 [79] 4051553674 2685248232 3585960433 5025685098 4720129137 3997211464
 [85] 2115574348 2169971697 2225948297 2122185651 2021793943 2067678545
 [91] 1679165513 3191184887 3804249352 3427966641 3515590147 3568643357
 [97] 4048985434 3040120062 2669436017 2384052337 3713222545 3281024312
[103] 3157186225 2955837591 3095777630 3496342093 3464383216 2277764233
[109] 2664272579 3380457605 3475415569 3055723620 3109522281 3280998814
[115] 3011414204 2855005165 3348868792 2823966602 2565759475 2832651712
[121] 2593576996 2971165814 2624299082 3228829254 2584808157 2713511663
[127] 2825554428 3277015234 3076074401 3745258350 3169690945 2905141524

In [336]:
file.size(dsn_dt5$bam_tx_file)
head(dsn_dt5$bam_tx_file)


[1] 2921448877 2303928727 3227451051 3803043105 2947717403 3417174304
  [7] 5024570323 2829877905 2999980009 3426864897 5684992475 3714388978
 [13] 4592108057 3915637029 5330928297 4935091289 4619870751  921622708
 [19]  800287209 2324159449 1553121264 1534979631  859486815 2282238779
 [25] 2140463180 1397229791 3048898240 3438099952  999287871 4667206402
 [31] 3596406126 2671853952 2623850414 4169741986 3874493814 5001160208
 [37] 4617842972 3327855182 3108133347 4308320216 2813317546 2337007408
 [43] 4512007618 3230212154 3264062370 5736802614 4079981641 3623969058
 [49] 2631806629 4552512993 4243657610 4173934567 4188894058 2625858136
 [55] 1964939097 1864736617 1880828817 1787009432 1741188419 1860557886
 [61] 2029715565 1961651484 2116415981 2044884214 1950400683 1853826780
 [67] 1905330419 1823725136 1786060508 4115243904 4483453191 2961711317
 [73] 5430065854 4161514025 2404559454 3425535696 3249252128 2828807041
 [79] 4350608024 2111207078 3512994103 3873371280 4953245849 3195952686
 [85] 1740851021 1681749926 2202460571 1721733086 1892489968 2058514484
 [91] 1802959263 4611673528 6714911157 5114516537 5557886712 5536762490
 [97] 6405644614 3576042958 3492415901 3398481057 5819449404 4805321182
[103] 4390429098 4293448622 4830232726 5084602834 5718385741 3039322753
[109] 3480934953 4242876064 4461747254 4377602018 3675207273 4661787121
[115] 3865404554 3336009672 3721451919 3511168007 3079229740 3559149508
[121] 3725629851 3711113687 2832209136 4044756525 3649649405 3926031092
[127] 3945417695 4183163604 4241353672 4722299067 3721173584 3842190566

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC//CD141DC_normal_rep1/CD141DC_normal_rep1_Aligned.toTranscriptome.out.bam"          
[2] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC//BDCA2CD2NpDC_normal_rep1/BDCA2CD2NpDC_normal_rep1_Aligned.toTranscriptome.out.bam"
[3] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC//BDCA2CD2PpDC_normal_rep1/BDCA2CD2PpDC_normal_rep1_Aligned.toTranscriptome.out.bam"
[4] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC//CD1cDC_normal_rep1/CD1cDC_normal_rep1_Aligned.toTranscriptome.out.bam"            
[5] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC//CD141DC_normal_rep2/CD141DC_normal_rep2_Aligned.toTranscriptome.out.bam"          
[6] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/RNA_seq/Macraphge_Monocyte_DC//BDCA2CD2NpDC_normal_rep2/BDCA2CD2NpDC_normal_rep2_Aligned.toTranscriptome.out.bam"

In [339]:
GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF/hs1.ncbiRefSeq.gtf"
TE_GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF/T2T_CHM13_v2_rmsk_TE.gtf"
bin_tecount = '/home/whu78/conda3/envs/te/bin/'

dsn_dt5[, tecount_jobname := paste0('tecount_', assay_sample_name)]
dsn_dt5[, tecount_oo := paste0(assay_sample_dir, 'tecount_', assay_sample_name, '_out')]
dsn_dt5[, tecount_eo := paste0(assay_sample_dir, 'tecount_', assay_sample_name, '_err')]

dsn_dt5[, tecount_cmd := paste0('bsub -J ', tecount_jobname, ' -n 10 -o ', tecount_oo, ' -e ', tecount_eo, ' -P macmon -R "rusage[mem=5GB]" ')]
dsn_dt5[, tecount_cmd := paste0(tecount_cmd, ' -W 99:00')]
dsn_dt5[, tecount_cmd := paste0(tecount_cmd, ' " cd ', assay_sample_dir, ' && ', bin_tecount, 'TEcount --sortByPos --format BAM --mode multi -b ', bam_file)] 
dsn_dt5[, tecount_cmd := paste0(tecount_cmd, ' --GTF ', GTF_FILE, ' --TE ', TE_GTF_FILE, ' --project tecount "')] 

#write(dsn_dt5[count_file == 'Yes', tecount_cmd], file = paste0(base_dir, 'run_tecount.sh'))
write(dsn_dt5[, tecount_cmd], file = paste0(data_dir, 'run_macmon_tecount.sh'))


In [59]:
dsn_dt5$tecount_file[5]

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC/CD141DC_normal_rep2/tecount.cntTable"

In [33]:
table(sub('_.*', '', dsn_dt5[1:11, sample_name]))


BDCA2CD2NpDC BDCA2CD2PpDC      CD141DC       CD1cDC 
           3            3            3            2 

# TElocal

In [341]:
GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF/hs1.ncbiRefSeq.gtf"
#TE_GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF/T2T_CHM13_v2_rmsk_TE.gtf"
#TE_loc_GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/T2T-CHM13_v2_rmsk_TE.gtf.locInd"
#TE_loc_GTF_FILE="/research/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/data/data/T2T_CHM13_v2_rmsk_TE.gtf.locInd"

TE_GTF_FILE = '/research/groups/jxugrp/home/common/Lab_Members/SyedAhmad/Projects_Main_Farhan/trimmed_copy_leukemia_samples/testig_new_T2T_GTF/T2T_CHM13_v2_rmsk_TE.gtf'
TE_loc_GTF_FILE = '/research/groups/jxugrp/home/common/Data_Public/RNA_seq/Blueprint_EGA_Data_EGAD00001000745_Transcriptional_Diversity_Human_Blood_progenitors_Chen_etal_Science_2014/T2T_CHM13_v2_rmsk_TE.gtf.locInd'

sif_telocal = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/images/telocal.sif'
bin_telocal = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/conda3/envs/snap/bin/'
exe_telocal = '/home/whu78/repos/TElocal/TElocal'
exe_telocal = '/home/whu78/conda3/envs/te/bin/TElocal'

dsn_dt5[, telocal_jobname := paste0('telocal_', assay_sample_name)]
dsn_dt5[, telocal_oo := paste0(assay_sample_dir, 'telocal_', assay_sample_name, '_out')]
dsn_dt5[, telocal_eo := paste0(assay_sample_dir, 'telocal_', assay_sample_name, '_err')]

dsn_dt5[, telocal_cmd := paste0('bsub -J ', telocal_jobname, ' -n 10 -o ', telocal_oo, ' -e ', telocal_eo, ' -P macmon -R "rusage[mem=5GB]" ')]
dsn_dt5[, telocal_cmd := paste0(telocal_cmd, ' -W 99:00')]
dsn_dt5[, telocal_cmd := paste0(telocal_cmd, ' " cd ', assay_sample_dir, ' && ', exe_telocal, ' --sortByPos -b ', bam_file)] 
dsn_dt5[, telocal_cmd := paste0(telocal_cmd, ' --GTF ', GTF_FILE, ' --TE ', TE_loc_GTF_FILE, ' --stranded reverse --project telocal "')] 
#dsn_dt5[, telocal_cmd := paste0(telocal_cmd, ' --outdir ', telocal_dir)]

write(dsn_dt5[, telocal_cmd], file = paste0(data_dir, 'run_macmon_telocal.sh'))


In [342]:
dsn_dt5[, tecount_file := paste0(assay_sample_dir, 'tecount.cntTable')] 

In [344]:
file.size(dsn_dt5$tecount_file)

[1] 819055 818831 821395 822356 818922 822456 826658 820012 819814 823163
 [11] 828424 821825 824891 822813 828493 828733 829349 797304 799669 803839
 [21] 808675 805095 797873 804364 812285 809228 814112 816222 803408 821063
 [31] 820747 818189 817638 821898 817281 823856 822946 820048 816032 819014
 [41] 815392 814011 818461 816258 818190 824838 821524 820635 817942 824044
 [51] 820774 823550 818575 816011 810998 810477 810640 810336 810226 813969
 [61] 814873 814611 815627 814255 813879 813549 813576 812936 808901 822016
 [71] 822860 817878 823876 821092 816165 820648 821139 818720 824784 815839
 [81] 820052 824757 823736 819347 815735 813991 816263 812759 812641 811688
 [91] 811294 822946 824176 824008 823910 823774 825592 821412 821470 819882
[101] 824507 822401 822900 822656 824233 825133 825793 819088 819937 822992
[111] 823517 822276 821827 824132 819870 819221 820479 817640 821470 821716
[121] 822703 823354 821266 820659 819549 821670 821933 823176 821967 824172
[131] 821765 822001

# import TEcount

In [369]:
coding_genes = read.table('/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/coding_gene_list', header = F)$V1
te_genes = read.table('/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/collab_Farhan/te_gene_list', header = F)$V1


In [345]:
tecount_list = lapply(dsn_dt5$tecount_file, function(xx){
    tmp = fread(xx)
    setnames(tmp, c('gene', 'count'))
})

In [347]:
names(tecount_list) = dsn_dt5$assay_sample_name
tecount_dt = rbindlist(tecount_list, idcol = T) 
tecount_dt = dcast(gene ~ .id, value.var = 'count', data = tecount_dt)
tecount_dt[1:10, 1:10]


gene,BDCA2CD2NpDC_normal_rep1,BDCA2CD2NpDC_normal_rep2,BDCA2CD2NpDC_normal_rep3,BDCA2CD2PpDC_normal_rep1,BDCA2CD2PpDC_normal_rep2,BDCA2CD2PpDC_normal_rep3,CD141DC_normal_rep1,CD141DC_normal_rep2,CD141DC_normal_rep3
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
(CATTC)n:Satellite:Satellite,0,1,0,1,0,0,0,0,0
(GAATG)n:Satellite:Satellite,3,1,4,7,6,17,2,1,0
7SLRNA:srpRNA:srpRNA,0,0,1,1,0,1,0,0,3
A-rich:Low_complexity:Low_complexity,0,0,0,0,0,0,0,0,0
A1BG,322,487,472,176,597,100,84,13,76
A1BG-AS1,98,145,156,68,212,196,47,63,41
A1CF,0,0,3,0,2,12,0,0,0
A2M,13,37,24,74,53,73,26,37,38
A2M-AS1,36,107,179,35,164,250,3,12,24


In [348]:
tecount_df = setDF(tecount_dt[, 2:ncol(tecount_dt)], rownames = tecount_dt$gene)


In [351]:
fwrite(tecount_df, file = 'tecount_assay_sample_mtx.tsv')

In [354]:
tecount_df_t = as.data.table(t(tecount_df), keep.rownames = T)
tecount_df_t = merge(tecount_df_t, dsn_dt5[, .(assay_sample_name, sample_name)], by.x = 'rn', by.y = 'assay_sample_name')
head(tecount_df_t)

rn,(CATTC)n:Satellite:Satellite,(GAATG)n:Satellite:Satellite,7SLRNA:srpRNA:srpRNA,A-rich:Low_complexity:Low_complexity,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,⋯,hAT-1_Mam:hAT-Tag1:DNA,hAT-4b_Ther:hAT-Tip100:DNA,hAT-5_Mam:hAT-Tag1:DNA,hAT-N1_Mam:hAT-Tip100:DNA,hAT-N1_Mam:hAT-Tip100?:DNA,hAT-N1a_Mam:hAT-Tip100:DNA,teucerv1_5edge:Unknown:Unknown,teucerv2_3edge:Unknown:Unknown,teucerv3_internal:Unknown:Unknown,sample_name
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
BDCA2CD2NpDC_normal_rep1,0,3,0,0,322,98,0,13,36,⋯,442,57,213,38,0,117,108,164,1,BDCA2CD2NpDC_normal_rep1
BDCA2CD2NpDC_normal_rep2,1,1,0,0,487,145,0,37,107,⋯,655,72,352,64,0,163,278,294,3,BDCA2CD2NpDC_normal_rep2
BDCA2CD2NpDC_normal_rep3,0,4,1,0,472,156,3,24,179,⋯,691,79,366,52,0,162,395,340,0,BDCA2CD2NpDC_normal_rep3
BDCA2CD2PpDC_normal_rep1,1,7,1,0,176,68,0,74,35,⋯,571,58,403,63,0,189,122,189,2,BDCA2CD2PpDC_normal_rep1
BDCA2CD2PpDC_normal_rep2,0,6,0,0,597,212,2,53,164,⋯,1032,141,594,67,0,284,392,405,1,BDCA2CD2PpDC_normal_rep2
BDCA2CD2PpDC_normal_rep3,0,17,1,0,100,196,12,73,250,⋯,1758,197,1050,131,0,451,74,309,0,BDCA2CD2PpDC_normal_rep3


In [355]:
tecount_df_t[, rn := NULL]
tecount_df_t = tecount_df_t %>% group_by(sample_name) %>% across(list(sum))


Warning message:
"`summarise_each()` was deprecated in dplyr 0.7.0.
ℹ Please use `across()` instead."


In [363]:
tecount_df_t = as.data.table(tecount_df_t)

In [364]:
head(tecount_df_t)

sample_name,(CATTC)n:Satellite:Satellite,(GAATG)n:Satellite:Satellite,7SLRNA:srpRNA:srpRNA,A-rich:Low_complexity:Low_complexity,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,⋯,bA255A11.4,hAT-1_Mam:hAT-Tag1:DNA,hAT-4b_Ther:hAT-Tip100:DNA,hAT-5_Mam:hAT-Tag1:DNA,hAT-N1_Mam:hAT-Tip100:DNA,hAT-N1_Mam:hAT-Tip100?:DNA,hAT-N1a_Mam:hAT-Tip100:DNA,teucerv1_5edge:Unknown:Unknown,teucerv2_3edge:Unknown:Unknown,teucerv3_internal:Unknown:Unknown
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
BDCA2CD2NpDC_normal_rep1,0,3,0,0,322,98,0,13,36,⋯,0,442,57,213,38,0,117,108,164,1
BDCA2CD2NpDC_normal_rep2,1,1,0,0,487,145,0,37,107,⋯,0,655,72,352,64,0,163,278,294,3
BDCA2CD2NpDC_normal_rep3,0,4,1,0,472,156,3,24,179,⋯,0,691,79,366,52,0,162,395,340,0
BDCA2CD2PpDC_normal_rep1,1,7,1,0,176,68,0,74,35,⋯,0,571,58,403,63,0,189,122,189,2
BDCA2CD2PpDC_normal_rep2,0,6,0,0,597,212,2,53,164,⋯,0,1032,141,594,67,0,284,392,405,1
BDCA2CD2PpDC_normal_rep3,0,17,1,0,100,196,12,73,250,⋯,0,1758,197,1050,131,0,451,74,309,0


In [366]:
tecount_df_t_df = setDF(tecount_df_t[, 2:ncol(tecount_df_t)], rownames = tecount_df_t$sample_name)
tecount_df_t_df = t(tecount_df_t_df)
head(tecount_df_t_df)                                

,BDCA2CD2NpDC_normal_rep1,BDCA2CD2NpDC_normal_rep2,BDCA2CD2NpDC_normal_rep3,BDCA2CD2PpDC_normal_rep1,BDCA2CD2PpDC_normal_rep2,BDCA2CD2PpDC_normal_rep3,CD141DC_normal_rep1,CD141DC_normal_rep2,CD141DC_normal_rep3,CD1cDC_normal_rep2,⋯,Monocyte_BRCA_rep3,Monocyte_CaEnd_rep1,Monocyte_CaEnd_rep2,Monocyte_CaEnd_rep3,Monocyte_normal_rep1,Monocyte_normal_rep2,Monocyte_normal_rep3,Monocyte_normal_rep4,Monocyte_normal_rep5,Monocyte_normal_rep6
(CATTC)n:Satellite:Satellite,0,1,0,1,0,0,0,197,0,3511,⋯,32246,60,0,0,0,240,96,1,23,112
(GAATG)n:Satellite:Satellite,3,1,4,7,6,17,2,384,0,7146,⋯,25446,46,6,4,0,435,77,3,18,186
7SLRNA:srpRNA:srpRNA,0,0,1,1,0,1,0,73,3,28,⋯,34,11,54,2,0,107,7,151,239,347
A-rich:Low_complexity:Low_complexity,0,0,0,0,0,0,0,4,0,11,⋯,0,3,4,0,0,1,0,0,10,18
A1BG,322,487,472,176,597,100,84,167,76,435,⋯,381,13,317,84,0,490,42,1377,1220,1591
A1BG-AS1,98,145,156,68,212,196,47,140,41,255,⋯,194,18,109,0,1,119,15,700,563,597


In [367]:
tecount_sample_df = tecount_df_t_df

In [370]:
tecount_sample_coding_df = tecount_sample_df[rownames(tecount_sample_df) %in% coding_genes, ] 
tecount_sample_te_df = tecount_sample_df[rownames(tecount_sample_df) %in% te_genes, ] 

In [368]:
rm(tecount_df_t, tecount_df_t_df)

# export 

In [ ]:
fwrite(dsn_dt5, file = 'dsn_dt5.tsv')
fwrite(tecount_df, 

In [385]:
dsn_sample = dsn_dt5[!duplicated(sample_name), ]
fwrite(dsn_sample, file = 'dsn_sample.tsv', sep = '\t')

In [379]:
write.table(tecount_sample_df, file = 'tecount_sample_df.tsv', sep = '\t') 
write.table(tecount_sample_coding_df, file = 'tecount_sample_coding_df.tsv', sep = '\t')
write.table(tecount_sample_te_df, file = 'tecount_sample_te_df.tsv', sep = '\t')


In [377]:
norm = colnames(tecount_sample_df)[grep('norm|Breast|Endo', colnames(tecount_sample_df))] 
norm

[1] "BDCA2CD2NpDC_normal_rep1"    "BDCA2CD2NpDC_normal_rep2"   
 [3] "BDCA2CD2NpDC_normal_rep3"    "BDCA2CD2PpDC_normal_rep1"   
 [5] "BDCA2CD2PpDC_normal_rep2"    "BDCA2CD2PpDC_normal_rep3"   
 [7] "CD141DC_normal_rep1"         "CD141DC_normal_rep2"        
 [9] "CD141DC_normal_rep3"         "CD1cDC_normal_rep2"         
[11] "CD1cDC_normal_rep3"          "Macrophage_Breast_rep1"     
[13] "Macrophage_Breast_rep2"      "Macrophage_Endometrial_rep1"
[15] "Monocyte_normal_rep1"        "Monocyte_normal_rep2"       
[17] "Monocyte_normal_rep3"        "Monocyte_normal_rep4"       
[19] "Monocyte_normal_rep5"        "Monocyte_normal_rep6"

In [383]:
getwd()

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/Macraphge_Monocyte_DC"

In [389]:
dsn_sample[, cell_type := unlist(strsplit(sample_name, '_'))[1], by = 1:nrow(dsn_sample)]


In [390]:
head(dsn_sample[, .(sample_name, cell_type)])

sample_name,cell_type
<chr>,<chr>
BDCA2CD2NpDC_normal_rep1,BDCA2CD2NpDC
BDCA2CD2NpDC_normal_rep2,BDCA2CD2NpDC
BDCA2CD2NpDC_normal_rep3,BDCA2CD2NpDC
BDCA2CD2PpDC_normal_rep1,BDCA2CD2PpDC
BDCA2CD2PpDC_normal_rep2,BDCA2CD2PpDC
BDCA2CD2PpDC_normal_rep3,BDCA2CD2PpDC


In [391]:
write.table(tecount_sample_df[, norm], file = 'tecount_sample_normal_df.tsv', sep = '\t') 
setkey(dsn_sample, 'sample_name')
write.table(dsn_sample[norm, ], file = 'dsn_sample_normal.tsv', sep = '\t')
